In [0]:
import pandas as pd
import requests
import json

In [0]:
df = pd.read_csv('https://s3.amazonaws.com/vargo.aprd6342/data/msas.csv')
df.head()
#From info in assignment page: parameter #1 should be 50k - 74.9k HH income: B19001
#parameter #2 should be ages 18-34: B01001
#parameter #3 should we target areas with large hispanic pop? Hispanic: B03002

,CBSA,NAME,POPESTIMATE2017
0,10180,"Abilene, TX",170219
1,10420,"Akron, OH",703505
2,10500,"Albany, GA",151434
3,10540,"Albany, OR",125047
4,10580,"Albany-Schenectady-Troy, NY",886188


In [0]:
#add the geo id# to df
tableids = ['B19001', 'B01001', 'B03002']
tablestring = ",".join(tableids)

geoid = '31000US' + df['CBSA'].astype(str)
df['geoid'] = geoid
df.head()

,CBSA,NAME,POPESTIMATE2017,geoid
0,10180,"Abilene, TX",170219,31000US10180
1,10420,"Akron, OH",703505,31000US10420
2,10500,"Albany, GA",151434,31000US10500
3,10540,"Albany, OR",125047,31000US10540
4,10580,"Albany-Schenectady-Troy, NY",886188,31000US10580


In [0]:
#pull data from api

#requesturl = 'https://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=310|01000US' % tablestring
#tableidstring = ','.join(tableids)
#loadedjson = requests.get(url)
#parsedjson = loadedjson.json()

#got error doing it all at once. Breaking into two parts then combining works. 
geoidstring1 = ",".join(geoid[:int(len(geoid)/2)])
requesturl1 = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=%s' % (tableidsstring, geoidstring1)
loadedjson1 = requests.get(requesturl1)
parsedjson1 = json.loads(loadedjson1.text)

#second part
geoidstring2 = ",".join(geoid[int(len(geoid)/2):])
requesturl2 = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=%s' % (tableidsstring, geoidstring2)
loadedjson2 = requests.get(requesturl2)
parsedjson2 = json.loads(loadedjson2.text)

#combine the two
alldata = dict(parsedjson1['data'], **parsedjson2['data'])
alldata['31000US10180']['B03002']['estimate']

{'B03002001': 174006.0,
 'B03002002': 133580.0,
 'B03002003': 110633.0,
 'B03002004': 13466.0,
 'B03002005': 648.0,
 'B03002006': 3491.0,
 'B03002007': 0.0,
 'B03002008': 578.0,
 'B03002009': 4764.0,
 'B03002010': 706.0,
 'B03002011': 4058.0,
 'B03002012': 40426.0,
 'B03002013': 21001.0,
 'B03002014': 1075.0,
 'B03002015': 0.0,
 'B03002016': 461.0,
 'B03002017': 27.0,
 'B03002018': 16003.0,
 'B03002019': 1859.0,
 'B03002020': 871.0,
 'B03002021': 988.0}

In [0]:
#setting our parameters
HHincome = []
Popbyage = []
PopHispanic = []
for i, row in df.iterrows():
    Income = alldata[df['geoid'][i]]['B19001']['estimate']['B19001011']#  $50,000 to $59,999
    + alldata[df['geoid'][i]]['B19001']['estimate']['B19001012'] #  $60,000 to $74,999
    HHincome.append(Income)
    Pop = alldata[df['geoid'][i]]['B01001']['estimate']['B01001007']#  18 and 19 Years
    + alldata[df['geoid'][i]]['B01001']['estimate']['B01001008']#  20 Years
    + alldata[df['geoid'][i]]['B01001']['estimate']['B01001009']#  21 Years
    + alldata[df['geoid'][i]]['B01001']['estimate']['B01001010']#  22 to 24 Years
    + alldata[df['geoid'][i]]['B01001']['estimate']['B01001011']#  25 to 29 Years
    + alldata[df['geoid'][i]]['B01001']['estimate']['B01001012']#  30 to 34 Years
    
    Popbyage.append(Pop)
    Hispanic = alldata[df['geoid'][i]]['B03002']['estimate']['B03002012']#  Hispanic or Latino
    PopHispanic.append(Hispanic)

df['Pop Income'] = HHincome
df['Pop by Age'] = Popbyage
df['Pop Hispanic'] = PopHispanic

In [0]:
#Finding which city has the highest count of the audience for parameter #1: 50k - 74.9k HH income: B19001
print(df.loc[df['Pop Income'] == max(df['Pop Income'])]['NAME'])

249    New York-Newark-Jersey City, NY-NJ-PA
Name: NAME, dtype: object


#1
New York-Newark-Jersey City has the highest count of the audience with a household income of 50-74.9K 

In [0]:
#Finding which city has the highest count of the audience for parameter #2: ages 18-34: B01001
print(df.loc[df['Pop by Age'] == max(df['Pop by Age'])]['NAME'])

249    New York-Newark-Jersey City, NY-NJ-PA
Name: NAME, dtype: object


#2
New York-Newark-Jersey City has the highest count of the audience with ages 18-34 years old

In [0]:
#Finding which city has the highest count of the audience for parameter #3: Hispanic Consumers
print(df.loc[df['Pop Hispanic'] == max(df['Pop Hispanic'])]['NAME'])

210    Los Angeles-Long Beach-Anaheim, CA
Name: NAME, dtype: object


#3
Los Angeles-Long Beach-Anaheim, CA has the highest count of the audience with a greater population of Hispanics

In [0]:
#What city has the highest mean percentage across all three parameters? 
#convert each parameter to percentages. Each parameter % will simply be 
#the population count for that parameter / the highest population count for that parameter across all MSAs
HHincomepercent = []
Popbyagepercent = []
PopHispanicpercent = []
for i, row in df.iterrows():
    Incomepercent = df['Pop Income'][i]/max(df['Pop Income'])
    Agepercent = df['Pop by Age'][i]/max(df['Pop by Age'])
    Hispanicpercent = df['Pop Hispanic'][i]/max(df['Pop Hispanic'])
    HHincomepercent.append(Incomepercent)
    Popbyagepercent.append(Agepercent)
    PopHispanicpercent.append(Hispanicpercent)

df['HH Income %'] = HHincomepercent
df['Pop by Age %'] = Popbyagepercent
df['Pop Hispanic %'] = PopHispanicpercent

mean_percentage = []
for i, row in df.iterrows():
    Mean = (df['HH Income %'][i] + df['Pop by Age %'][i]+ df['Pop Hispanic %'][i])/3
    mean_percentage.append(Mean)
df['mean_percentage'] = mean_percentage

#finding the city with the highest percentage across all parameters 
print(df.loc[df['mean_percentage'] == max(df['mean_percentage'])]['NAME'])


249    New York-Newark-Jersey City, NY-NJ-PA
Name: NAME, dtype: object


#4
New York-Newark-Jersey City, NY-NJ-PA has the highest percentage of the audience across all three parameters. 